In [0]:
%python
dim_atleta = 'workspace.project_data_football_silver.dim_atleta'
dim_campeonato = 'workspace.project_data_football_silver.dim_campeonato'
dim_clube = 'workspace.project_data_football_silver.dim_clube'
dim_estadio = 'workspace.project_data_football_silver.dim_estadio'
dim_posicao = 'workspace.project_data_football_silver.dim_posicao'
dim_rodada = 'workspace.project_data_football_silver.dim_rodada'
fato_partida = 'workspace.project_data_football_silver.fato_partida'
fato_pontuacao = 'workspace.project_data_football_silver.fato_pontuacao'

In [0]:
%python
spark.table(dim_rodada).createOrReplaceTempView("tmp_silver_rodada")
spark.table(fato_partida).createOrReplaceTempView("tmp_silver_partida")
spark.table(dim_estadio).createOrReplaceTempView("tmp_silver_estadio")
spark.table(dim_clube).createOrReplaceTempView("tmp_silver_clube")
spark.table(dim_campeonato).createOrReplaceTempView("tmp_silver_campeonato")

In [0]:
describe tmp_silver_partida

In [0]:
describe tmp_silver_estadio

In [0]:

-- Métricas por Temporada + Rodada

WITH base_partidas AS (
  -- Base enriquecida (fato + dimensões)
  SELECT
    f.partida_id, -- ID partida
    f.gols_casa, -- Gols mandante
    f.gols_visitante, -- Gols visitante
    f.rodada_id, -- FK rodada
    f.estadio_id, -- FK estadio
    c.temporada, -- Ano campeonato
    r.numero_rodada -- Nº rodada
  FROM
    tmp_silver_partida f
      JOIN tmp_silver_rodada r
        ON f.rodada_id = r.rodada_id
      JOIN tmp_silver_campeonato c
        ON f.campeonato_id = c.campeonato_id
),
gols_por_rodada AS (
  -- Agregado por rodada
  SELECT
    temporada, -- Ano
    numero_rodada, -- Rodada
    COUNT(*) AS total_partidas, -- Qtde jogos
    SUM(gols_casa + gols_visitante) AS total_gols_rodada, -- Gols rodada
    ROUND(AVG(gols_casa + gols_visitante), 1) AS media_gols_jogos -- Média gols
  FROM
    base_partidas
  GROUP BY
    temporada,
    numero_rodada
),
mandante_vs_visitante AS (
  -- Resultado geral da temporada
  SELECT
    temporada, -- Ano
    SUM(
      CASE
        WHEN gols_casa > gols_visitante THEN 1
        ELSE 0
      END
    ) AS vitorias_mandante, -- Vitórias casa
    SUM(
      CASE
        WHEN gols_casa < gols_visitante THEN 1
        ELSE 0
      END
    ) AS vitorias_visitante, -- Vitórias fora
    SUM(
      CASE
        WHEN gols_casa = gols_visitante THEN 1
        ELSE 0
      END
    ) AS empates, -- Empates
    COUNT(*) AS total_partidas_temporada -- Total jogos temporada
  FROM
    base_partidas
  GROUP BY
    temporada
),
evolucao_temporada AS (
  -- Total de gols por temporada
  SELECT
    temporada, -- Ano
    SUM(total_gols_rodada) AS total_gols_temporada -- Gols totais ano
  FROM
    gols_por_rodada
  GROUP BY
    temporada
)

-- SELECT FINAL - KPI consolidado

SELECT
  g.temporada, -- Ano
  g.numero_rodada, -- Rodada
  g.total_partidas, -- Jogos rodada
  g.total_gols_rodada, -- Gols rodada
  g.media_gols_jogos, -- Média gols rodada
  -- % vitória mandante
  ROUND(100.0 * m.vitorias_mandante / m.total_partidas_temporada, 2) AS perc_vitoria_mandante,
  -- % vitória visitante
  ROUND(100.0 * m.vitorias_visitante / m.total_partidas_temporada, 2) AS perc_vitoria_visitante,
  -- Gols acumulado (window function)
  SUM(g.total_gols_rodada) OVER (
      PARTITION BY g.temporada
      ORDER BY g.numero_rodada
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS gols_acumulado_temporada,
  e.total_gols_temporada -- Total gols temporada
FROM
  gols_por_rodada g
    JOIN mandante_vs_visitante m
      ON g.temporada = m.temporada
    JOIN evolucao_temporada e
      ON g.temporada = e.temporada
ORDER BY
  g.temporada,
  g.numero_rodada;

In [0]:
%python
df_gold_kpi_partidas = spark.sql("""

WITH base_partidas AS (
  SELECT
    f.partida_id,
    f.gols_casa,
    f.gols_visitante,
    f.rodada_id,
    f.estadio_id,
    c.temporada,
    r.numero_rodada
  FROM
    tmp_silver_partida f
      JOIN tmp_silver_rodada r
        ON f.rodada_id = r.rodada_id
      JOIN tmp_silver_campeonato c
        ON f.campeonato_id = c.campeonato_id
),
gols_por_rodada AS (
  SELECT
    temporada,
    numero_rodada,
    COUNT(*) AS total_partidas,
    SUM(gols_casa + gols_visitante) AS total_gols_rodada,
    ROUND(AVG(gols_casa + gols_visitante), 1) AS media_gols_jogos
  FROM
    base_partidas
  GROUP BY
    temporada,
    numero_rodada
),
mandante_vs_visitante AS (
  SELECT
    temporada,
    SUM(
      CASE
        WHEN gols_casa > gols_visitante THEN 1
        ELSE 0
      END
    ) AS vitorias_mandante,
    SUM(
      CASE
        WHEN gols_casa < gols_visitante THEN 1
        ELSE 0
      END
    ) AS vitorias_visitante,
    SUM(
      CASE
        WHEN gols_casa = gols_visitante THEN 1
        ELSE 0
      END
    ) AS empates,
    COUNT(*) AS total_partidas_temporada
  FROM
    base_partidas
  GROUP BY
    temporada
),
evolucao_temporada AS (
  SELECT
    temporada,
    SUM(total_gols_rodada) AS total_gols_temporada
  FROM
    gols_por_rodada
  GROUP BY
    temporada
)
SELECT
  g.temporada,
  g.numero_rodada,
  g.total_partidas,
  g.total_gols_rodada,
  g.media_gols_jogos,
  ROUND(100.0 * m.vitorias_mandante / m.total_partidas_temporada, 2) AS perc_vitoria_mandante,
  ROUND(100.0 * m.vitorias_visitante / m.total_partidas_temporada, 2) AS perc_vitoria_visitante,
  SUM(g.total_gols_rodada) OVER (
      PARTITION BY g.temporada
      ORDER BY g.numero_rodada
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS gols_acumulado_temporada,
  e.total_gols_temporada
FROM
  gols_por_rodada g
    JOIN mandante_vs_visitante m
      ON g.temporada = m.temporada
    JOIN evolucao_temporada e
      ON g.temporada = e.temporada
ORDER BY
  g.temporada,
  g.numero_rodada;
""")

#salvar em delta e salvar tabela
df_gold_kpi_partidas.write\
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.project_data_football_gold.kpi_partidas")

In [0]:
SELECT * FROM workspace.project_data_football_gold.kpi_partidas